In [11]:
from config import *
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
import torch.nn.functional as F
import os
from data.data_loader import *
from model import *
import torch.optim as optim
from  RetrievalTest import *

batchSize = 4

print("Loading the dataset")
Source_x, Source_y, Target_x = prepare_Data(data_dir, True)
Gallery_x, Query_x = prepare_Data(data_dir, False)
similarity = csr_matrix(scipy.io.loadmat("data/cifar10/cifar10_Similarity.mat")['label_Similarity']).todense()

print("Data loading finished")

gallery = torch.utils.data.DataLoader(Gallery_x,batch_size=1000)
query = torch.utils.data.DataLoader(Query_x,batch_size=1000)

source = torch.utils.data.DataLoader([(Source_x[i], Source_y[i]) for i in range(len(Source_x))],batch_size=batchSize, shuffle=True)
target = torch.utils.data.DataLoader(Target_x,batch_size=batchSize,shuffle=True)



Loading the dataset
--------------------Loading dataset--------------------
Source:  (5000, 32, 32, 3) (5000,)
Target:  (54000, 32, 32, 3)
--------------------Loading dataset--------------------
Gallery:  (54000, 32, 32, 3) (54000,)
Query:  (1000, 32, 32, 3) (1000,)
Data loading finished


In [2]:
vggModel=models.vgg16_bn(pretrained=True)
# vggModel.children

net1, net2 = [], []
for i in vggModel.children():
    for r, i in enumerate(i.children()):
        if r <=23: net1.append(i)
        elif r<= 32: net2.append(i)
    break
net1, net2  = nn.Sequential(*net1), nn.Sequential(*net2)

In [3]:
def intranorm(features,n_book):
    x = features.split(n_book,1)
    
    for b in range(n_book):
        if b==0: dummy = F.normalize(x[b],1)
        else:
            dummy = torch.cat((dummy,F.normalize(x[b],1)),1)
    return dummy

def shape_(inp):
     for i in inp:
            print(f"shape is: {i.shape}")
            
def Indexing(Z,des,numSeg):
        Z = intranorm(Z,numSeg)
        x = torch.split(des,numSeg,1)
        y = torch.split(Z,numSeg,1)
        for i in range(numSeg):
            size_x = x[i].shape[0]
            size_y = y[i].shape[0]
            xx = x[i].unsqueeze(-1)

            dummy = torch.tensor(1)
            xx = xx.tile([1,1,size_y])
            yy = y[i].unsqueeze(-1)
            yy = yy.tile([1,1,size_x])
            yy = yy.permute(2,1,0)
            diff = torch.sum(torch.multiply(xx,yy),1)

            arg = torch.argmax(diff,1)
            max_idx = arg.reshape(-1,1)

            if i == 0: quant_idx = max_idx
            else: quant_idx = torch.cat((quant_idx,max_idx),1)

        return quant_idx

In [4]:
class softassignment_(nn.Module):
    def __init__(self,len_code, n_book, intn_word):
        super(softassignment_,self).__init__()
        self.Z = nn.Linear(len_code * n_book,intn_word, bias=False)

    def forward(self,features,n_book,alpha,):
        z_ = intranorm(self.Z.state_dict()['weight'], n_book).split(n_book,1)
        x_ = features.split(n_book,1)

        for i in range(n_book):
            size_z = z_[i].shape[0] # number of codewords
            size_x = x_[i].shape[0] # batch size
            xx = x_[i].unsqueeze(-1)
            xx = xx.repeat(1,1,size_z)
            zz = z_[i].unsqueeze(-1)
            zz = zz.repeat(1,1,size_x).T

            diff = 1 - torch.sum(torch.mul(xx,zz), 1) # 32,16
            softmax_diff = F.softmax(diff*(-alpha),1) #32,16
            soft_des_temp = torch.matmul(softmax_diff,z_[i]) # 32,12
            if i == 0: descriptor = soft_des_temp
            else: descriptor = torch.cat((descriptor,soft_des_temp),1)

        return intranorm(descriptor,n_book) # 32,144

In [5]:
class classifier_(nn.Module):
    def __init__(self,n_CLASSES, len_code, n_book,):
        super(classifier_,self).__init__()
        self.prototypes = nn.Linear(len_code * n_book, n_CLASSES, bias=False)
        self.n_book = n_book
        
    def forward(self, x):
        x_ = x.split(self.n_book,1)
        c_ = (intranorm(self.prototypes.state_dict()['weight'], n_book)*beta).T.split(self.n_book,0)
        for i in range(self.n_book):
            sub_res = torch.matmul(x_[i], c_[i]).unsqueeze(-1)
            if i == 0: res = sub_res
            else: res = torch.cat((res,sub_res),2)
        
        return torch.sum(res, 2)

In [6]:
class features_(nn.Module):
    def __init__(self, net1, net2):
        super(features_,self).__init__()
        
        self.net1 = net1
        self.net2 = net2
        self.gavgp = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(768, len_code*n_book)
        
    def forward(self,x):
        x = self.net1(x) # shape: torch.Size([32, 3, 32,32])>torch.Size([32, 3, 4, 4])
        x_branch = self.gavgp(x)
        x = self.net2(x) # shape: torch.Size([32, 3, 4, 4])> torch.Size([32, 3, 4, 4])
        x = self.gavgp(x)
        
        x = torch.cat((x,x_branch),1)
        
        return self.linear(x.view(-1,768))
    
# x = torch.randn(32, 3, 32, 32, device='cpu')
# model = features_(net1, net2)
# out = model(x)# model
# out.shape, out.split(12,1)[0].shape, model.Z.shape, model.prototypes.shape

In [7]:
class flipGradient_(nn.Module):
    def forward(self,x,l=1.0):
        positivePath=(x*2).clone().detach().requires_grad_(False)
        negativePath=(-x).requires_grad_(True)
        return positivePath+negativePath


In [8]:
#models
model = features_(net1, net2).to(device)
classifier = classifier_(n_CLASSES, len_code, n_book).to(device)
softassignment = softassignment_(len_code, n_book, intn_word).to(device)
flipGradient = flipGradient_()
# optimizer
class_optim = optim.Adam(classifier.parameters(),lr=0.0002,betas=(0.5,0.999))
model_optim = optim.Adam(model.parameters(),lr=0.0002,betas=(0.5,0.999))
soft_optim = optim.Adam(softassignment.parameters(),lr=0.0002,betas=(0.5,0.999))

In [53]:
target_ = iter(target)
score = 0
save = ""
for epoch in tqdm(range(1)):
    m = 0
    model.train()
    classifier.train()
    softassignment.train()
    for df, batch in enumerate(source):
        x, y = batch
        x = torch.tensor(data_augmentation(x)).to(device)
        try: 
            xu = next(target_)
            xu = torch.tensor(data_augmentation(xu)).to(device)
        except:
            target_ = iter(target)
            xu = next(target_)
            xu = torch.tensor(data_augmentation(xu)).to(device)

        features = intranorm(model(x.permute(0,3,1,2)), n_book)
        featuresu = flipGradient(intranorm(model(xu.permute(0,3,1,2)), n_book))
        quanta = softassignment(features,n_book,alpha)
        logits = classifier(features* beta)
        
        y = y.to(device)
        
        cls_loss = torch.nn.functional.cross_entropy(logits,y)
        print(torch.nn.functional.cross_entropy(logits,y))
#         break
        y = torch.eye(numClasses)[y].to(device)
        entropy_loss = SMELoss(featuresu*beta ,intranorm(classifier.prototypes.state_dict()['weight'], n_book)*beta , n_book)
#         break
        y_ = torch.matmul(y,y.T)
#         y_ /= torch.sum(y_, axis=1, keepdims=True)
        break
        hash_loss = NPQLoss(y_,features, quanta,n_book)   
        
        final_loss = hash_loss + 0.1*entropy_loss + 0.1*cls_loss 
        m += final_loss.item()
        
        class_optim.zero_grad()
        model_optim.zero_grad()
        soft_optim.zero_grad()
        
        final_loss.backward()
        
        class_optim.step()
        model_optim.step()
        soft_optim.step()
        
        save += f"epoch: {df+1}/{epoch+1}\t{final_loss.item()}\t{hash_loss.item()}\t{cls_loss.item()}\t{entropy_loss.item()}\n"
        print(f"epoch: {df+1}/{epoch+1}\t{final_loss.item()}\t{hash_loss.item()}\t{cls_loss.item()}\t{entropy_loss.item()}\n")
       
    
    if False: #epoch % 50 == 0:
        
        with torch.no_grad():
            model.eval()
            classifier.eval()
            softassignment.eval()

            for r, i in tqdm(enumerate(gallery)):
                if r == 0: 
                    dummy = model(i.to(device).permute(0,3,1,2))
                else: 
                    dummy = torch.cat((dummy, model(i.to(device).permute(0,3,1,2))), 0)

            for r, i in tqdm(enumerate(query)):
                if r == 0: 
                    query_x = model(i.to(device).permute(0,3,1,2))
                else: 
                    query_x = torch.cat((query_x, model(i.to(device).permute(0,3,1,2))), 0)


        dummy = Indexing(intranorm(softassignment.Z.state_dict()['weight'].cpu(), n_book), dummy.cpu(), n_book)
        gallery_x = dummy.numpy().astype(int)
        quantizedDist = pqDist(intranorm(softassignment.Z.state_dict()['weight'].cpu(), n_book), n_book,gallery_x, query_x.cpu().numpy()).T
        Rank = np.argsort(quantizedDist, axis=0)
        mean_average_precision=cat_apcal(similarity,Rank,54000)
        save += f"epoch+score: {df+1}/{epoch+1}\t{final_loss.item()}\t{hash_loss.item()}\t{cls_loss.item()}\t{entropy_loss.item()}\t{mean_average_precision}\n"
        if mean_average_precision > score:
            score = mean_average_precision
            stateToBeSaved={
                'modelStateDict': [model.state_dict(),classifier.state_dict(), softassignment.state_dict()],
                'score': mean_average_precision,}
            torch.save(stateToBeSaved,f"{epoch}_copy_hemant.pth")
        print(mean_average_precision)
    with open("analysis_copy.txt", "w") as f:
        f.write(save)

tensor(1.6359, device='cuda:0', grad_fn=<NllLossBackward>)



In [55]:
y_.long()

tensor([[1, 0, 0, 1],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [1, 0, 0, 1]], device='cuda:0')

In [96]:
regAnchor=torch.mean(torch.sum(torch.square(features),dim=1))
regPositive=torch.mean(torch.sum(torch.square(quanta),dim=1))
l2Loss=torch.mul(1,regAnchor+regPositive)

In [97]:
l2Loss

tensor(3.0767, device='cuda:0', grad_fn=<MulBackward0>)

In [70]:
F.log_softmax(y_[0].view(1,-1),1)

tensor([[-1.0064, -2.0064, -2.0064, -1.0064]], device='cuda:0')

In [62]:
torch.log(y_)

tensor([[0., -inf, -inf, 0.],
        [-inf, 0., 0., -inf],
        [-inf, 0., 0., -inf],
        [0., -inf, -inf, 0.]], device='cuda:0')

In [61]:
y_ * torch.log(logits)

RuntimeError: The size of tensor a (4) must match the size of tensor b (10) at non-singleton dimension 1

In [ ]:
[0.1, 0.2, 0.7] (prediction) ------------------ [1.0, 0.0, 0.0] (target)

what you want is - (1.0 * log(0.1) + 0.0 * log(0.2) + 0.0 * log(0.7)) this is the cross entropy lossaa

In [58]:
torch.nn.functional.cross_entropy(y_,y_.long())

RuntimeError: multi-target not supported at /opt/conda/conda-bld/pytorch_1608538128634/work/aten/src/THCUNN/generic/ClassNLLCriterion.cu:15

In [ ]:
with torch.no_grad():
        model.eval()
        classifier.eval()
        softassignment.eval()

        for r, i in tqdm(enumerate(gallery)):
            if r == 0: 
                dummy = model(i.to(device).permute(0,3,1,2))
            else: 
                dummy = torch.cat((dummy, model(i.to(device).permute(0,3,1,2))), 0)

        for r, i in tqdm(enumerate(query)):
            if r == 0: 
                query_x = model(i.to(device).permute(0,3,1,2))
            else: 
                query_x = torch.cat((query_x, model(i.to(device).permute(0,3,1,2))), 0)


dummy = Indexing(intranorm(softassignment.Z.state_dict()['weight'].cpu(), n_book), dummy.cpu(), n_book)
gallery_x = dummy.numpy().astype(int)
quantizedDist = pqDist(intranorm(softassignment.Z.state_dict()['weight'].cpu(), n_book), n_book,gallery_x, query_x.cpu().numpy()).T
Rank = np.argsort(quantizedDist, axis=0)
mean_average_precision = cat_apcal(similarity,Rank,54000)
mean_average_precision

# THE END